<a href="https://colab.research.google.com/github/WildArcher/Master_work/blob/master/news_targeting_start(14-28.07).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LIBRARIES

In [2]:
import re
import requests
import bs4

При выкачивании текста новостей основная проблема была с кодировкой, которую никак не хотел преобразоввывать encode('cp1251'), после тщетных попыток пришлось написать алфавит в одной и другой кодировке

#WORK VERSION

In [170]:
url = requests.get('https://www.interfax.ru/business/718555') 
html = bs4.BeautifulSoup(url.text, "html.parser")
text = html.select('article')[0].getText()
#print(url_print.lower())

In [171]:
ISO_8859_1 = ['à', 'á', 'â', 'ã', 'ä', 'å', '¸', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 
              'ò', 'ó', 'ô', 'õ', 'ö', '÷', 'ø', 'ù', 'ú', 'û', 'ü', 'ý', 'þ', 'ÿ']

windows_1251 = ['а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с',
                'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']

header =  str(html.title.string) # заголовок новости
for i in range(33):
  text = text.lower().replace(ISO_8859_1[i], windows_1251[i])
  header = header.lower().replace(ISO_8859_1[i], windows_1251[i])

text = text.replace('\n', ' ')

In [70]:
#text #так выглядит текст новости

In [173]:
header

'рубль вечером ускорил снижение к доллару и евро на падении интереса к риску'

# A LOT OF PAGES

Делаем запрос "Сбербанк" на сайте https://www.interfax.ru/, находим новости, связанные со сбербанком на 13 страницах поиска за последний год, выкачиваем с этих 13 страниц ссылки на новости

In [176]:
time_interval = '17.07.2019&dt=21.07.2020' # с какого по какое ищем новости

page_1 ='https://www.interfax.ru/search/news/?sw=%F1%E1%E5%F0%E1%E0%ED%EA&df={}&sec=1447'.format(time_interval) # первая страница поиска

pages = [] # все страницы, которые содержат новости о сбербанке с 17.07.2019 по 17.07.2020
for i in range(14):
  pages.append(page_1 + '&p=page_{}'.format(i))
  
links = []  # список ссылок на всех страницах поиска
for page in pages:

  url = requests.get(page) 
  html = bs4.BeautifulSoup(url.text, "html.parser") # парсим страницу

  for link in html.find_all('a'):
    if ('/business/' in str(link)) and (len(str(link.get('href'))) == 16): # есть рекламные ссылки, но они большей длины, а длины 16 - только ссылки на нужные нам новости
      links.append(link.get('href'))

In [177]:
len(links) # не так много новостей, но можно изменить временной интервал с помощью time_interval

511

In [183]:
links[0] # так выглядит ссылка, полностью ссылка выглядит так: https://www.interfax.ru/business/718451

'/business/718451'

#TIME

In [247]:
url = requests.get('https://www.interfax.ru/business/718451') 
html = bs4.BeautifulSoup(url.text, "html.parser")

In [248]:
date = str(re.findall(r'article_publication_date: "..., .. ... .... ..:..:..', str(html)))

date = date.replace('article_publication_date: "', '')

weekday = date[2:5]
day = date[7:9]
month = date[10:13]
year = date[14:18]
hour = date[19:21]
minute = date[22:24]

In [258]:
date = str(re.findall(r'article_publication_date: "..., .+ ... .... .+:.+:..', str(html)))
date = date.replace('article_publication_date: "', '').replace('[', '').replace(']', '').replace(',', '').replace(r"'", '')
date

'Wed 22 Jul 2020 10:57:00'

# GATHERED PARTS

In [242]:
import pandas as pd
news = pd.DataFrame(columns = ['text', 'header', 'date'])

In [259]:
%%time
texts = []
headers = []
date = []

for link in links:

  url = requests.get('https://www.interfax.ru{}'.format(link)) 
  html = bs4.BeautifulSoup(url.text, "html.parser")
  text = html.select('article')[0].getText()

  date_ = str(re.findall(r'article_publication_date: "..., .+ ... .... ..:..:..', str(html)))
  date_ = date_.replace('article_publication_date: "', '').replace('[', '').replace(']', '').replace(',', '').replace(r"'", '')
  date.append(date_) # дата и время публикации новости

  header =  str(html.title.string) # заголовок новости
  for i in range(33):
    text = text.lower().replace(ISO_8859_1[i], windows_1251[i])
    header = header.lower().replace(ISO_8859_1[i], windows_1251[i])

  texts.append(text.replace('\n', ' '))
  headers.append(header)

CPU times: user 24.2 s, sys: 383 ms, total: 24.6 s
Wall time: 6min 58s


In [303]:
year = []
month = []
day_of_week = []
day = []
hour = []
minute = []

for i in range(news.shape[0]):

  hour.append(date[i][-8:-6])
  minute.append(date[i][-5:-3])
  day_of_week.append(date[i][:3])
  day.append(date[i][3:5])
  month.append(date[i][-17:-14])
  year.append(date[i][-13:-8])


In [304]:
news['day_of_week'] = day_of_week
news['day'] = day
news['month'] = month
news['year'] = year
news['minute'] = minute
news['hour'] = hour

In [ ]:
news.drop(columns=['date'], inplace=True)

In [310]:
news.head()

,text,header,day_of_week,day,month,year,minute,hour
0,"в россии выдано льготной ипотеки под 6,5% на ...","в россии выдано льготной ипотеки под 6,5% на 2...",Wed,2,Jul,2020,57,10
1,"сбербанк закрыл сделку по выкупу доли ""ßндекс...","сбербанк закрыл сделку по выкупу доли ""ßндекса...",Wed,2,Jul,2020,38,00
2,рынок акций рф во вторник вырос вслед за миро...,рынок акций рф во вторник вырос вслед за миром...,Tue,2,Jul,2020,17,19
3,рынок акций рф открылся ростом индексов мосби...,рынок акций рф открылся ростом индексов мосбир...,Tue,2,Jul,2020,35,10
4,"сбербанк выдал группе ""аэрофлот"" кредиты на 1...","сбербанк выдал группе ""аэрофлот"" кредиты на 1,...",Tue,2,Jul,2020,42,09


In [373]:
news.to_csv('news_2019-2020_sber_from_interfax.csv')

### Новости добыты, теперь нужно их разметить

In [1]:
import re
import pandas as pd
import datetime
import time

In [3]:
news = pd.read_csv('news_2019-2020_sber_from_interfax.csv')

In [4]:
news

,Unnamed: 0,text,header,day_of_week,day,month,year,minute,hour,target
0,0,"в россии выдано льготной ипотеки под 6,5% на ...","в россии выдано льготной ипотеки под 6,5% на 2...",Wed,2,Jul,2020,57,10,0
1,1,"сбербанк закрыл сделку по выкупу доли ""ßндекс...","сбербанк закрыл сделку по выкупу доли ""ßндекса...",Wed,2,Jul,2020,38,0,1
2,2,рынок акций рф во вторник вырос вслед за миро...,рынок акций рф во вторник вырос вслед за миром...,Tue,2,Jul,2020,17,19,1
3,3,рынок акций рф открылся ростом индексов мосби...,рынок акций рф открылся ростом индексов мосбир...,Tue,2,Jul,2020,35,10,1
4,4,"сбербанк выдал группе ""аэрофлот"" кредиты на 1...","сбербанк выдал группе ""аэрофлот"" кредиты на 1,...",Tue,2,Jul,2020,42,9,1
...,...,...,...,...,...,...,...,...,...,...
506,506,"en+ выкупит у втб 21,37% своих акций за $1,58...","en+ выкупит у втб 21,37% своих акций за $1,58 ...",Thu,6,Feb,2020,48,15,0
507,507,правительство и цб договорились о постоянных ...,правительство и цб договорились о постоянных к...,Wed,5,Feb,2020,10,17,1
508,508,"минэкономразвития запросило банки, как они мо...","минэкономразвития запросило банки, как они мог...",Wed,5,Feb,2020,22,12,0
509,509,государственно-частные партнерства выведут из...,государственно-частные партнерства выведут из-...,Wed,5,Feb,2020,58,9,0


In [71]:
#news.text[0]

## Возьмем котировки с финам с периодичностью 5 минут и час, чтобы отследить влияние новостей

In [5]:
def preprocessing_sber_data(sber):
  sber.columns=['date', 'time', 'open', 'high', 'low', 'close', 'vol']
  sber.time /= 10_000

  year, month, day = [], [], []
  for i in range(sber.shape[0]):
    year.append(time.strptime(str(sber.date[i]), '%Y%m%d')[0])
    month.append(time.strptime(str(sber.date[i]), '%Y%m%d')[1])
    day.append(time.strptime(str(sber.date[i]), '%Y%m%d')[2])

  sber['year'] = year
  sber['month'] = month
  sber['day'] = day

  #sber.drop(columns=['date'], inplace=True)

  return sber

sber_hour = preprocessing_sber_data(pd.read_csv('SBER_17.07.2019-2020_1hour.csv', sep=' '))

sber_5_min = preprocessing_sber_data(pd.read_csv('SBER_17.07.2019-2020_5min.csv', sep=' '))

In [6]:
sber_5_min.head()

,date,time,open,high,low,close,vol,year,month,day
0,20190717,10.05,234.32,237.00,234.02,236.96,1691860,2019,7,17
1,20190717,10.10,236.98,238.04,236.03,236.40,1802480,2019,7,17
2,20190717,10.15,236.40,236.71,235.83,235.96,780150,2019,7,17
3,20190717,10.20,235.96,236.13,235.38,235.91,590180,2019,7,17
4,20190717,10.25,235.91,236.55,235.81,236.41,543260,2019,7,17


In [7]:
date = []
for i in range(sber_5_min.shape[0]):
  date.append(datetime.datetime(sber_5_min.year[i], sber_5_min.month[i], sber_5_min.day[i], int(sber_5_min.time[i] // 1), int((sber_5_min.time[i] % 1)*101)))#

sber_5_min['date'] = date

date = []
for i in range(sber_hour.shape[0]):
  date.append(datetime.datetime(sber_hour.year[i], sber_hour.month[i], sber_hour.day[i], int(sber_hour.time[i] // 1)))

sber_hour['date'] = date

In [8]:
sber_hour['date'] = pd.to_datetime(sber_hour.date)

sber_5_min['date'] = pd.to_datetime(sber_5_min.date)

In [9]:
#date = []
#for i in range(sber_5_min.shape[0]):
#  date.append(datetime.datetime(2019, 7, 17, 10, 5, 00) + i*datetime.timedelta(0, 300))

#sber_5_min['date'] = date

In [10]:
sber_hour.head()

,date,time,open,high,low,close,vol,year,month,day
0,2019-07-17 11:00:00,11.0,234.32,238.04,234.02,237.12,8548250,2019,7,17
1,2019-07-17 12:00:00,12.0,237.12,237.43,236.55,237.20,2801380,2019,7,17
2,2019-07-17 13:00:00,13.0,237.20,237.20,236.81,237.18,2097860,2019,7,17
3,2019-07-17 14:00:00,14.0,237.17,237.70,236.70,237.19,2848650,2019,7,17
4,2019-07-17 15:00:00,15.0,237.19,237.58,236.85,237.13,1454710,2019,7,17


In [11]:
news.replace(to_replace=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], value=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], inplace=True)
news.drop(columns=['day_of_week'], inplace=True)

In [13]:
date = []
for i in range(news.shape[0]):
  date.append(datetime.datetime(news.year[i], news.month[i], news.day[i], news.hour[i], news.minute[i]))

news['date'] = date

In [72]:
news.head()

,Unnamed: 0,text,header,day,month,year,minute,hour,target,date
0,0,"в россии выдано льготной ипотеки под 6,5% на ...","в россии выдано льготной ипотеки под 6,5% на 2...",2,7,2020,57,10,0,2020-07-02 10:57:00
1,1,"сбербанк закрыл сделку по выкупу доли ""ßндекс...","сбербанк закрыл сделку по выкупу доли ""ßндекса...",2,7,2020,38,0,1,2020-07-02 00:38:00
2,2,рынок акций рф во вторник вырос вслед за миро...,рынок акций рф во вторник вырос вслед за миром...,2,7,2020,17,19,1,2020-07-02 19:17:00
3,3,рынок акций рф открылся ростом индексов мосби...,рынок акций рф открылся ростом индексов мосбир...,2,7,2020,35,10,1,2020-07-02 10:35:00
4,4,"сбербанк выдал группе ""аэрофлот"" кредиты на 1...","сбербанк выдал группе ""аэрофлот"" кредиты на 1,...",2,7,2020,42,9,1,2020-07-02 09:42:00


In [ ]:
open_before = np.zeros(shape=news.shape[0], dtype=float)
open_in_moment = np.zeros(shape=news.shape[0], dtype=float)
open_after = np.zeros(shape=news.shape[0], dtype=float)

close_before = np.zeros(shape=news.shape[0], dtype=float)
close_in_moment = np.zeros(shape=news.shape[0], dtype=float)
close_after = np.zeros(shape=news.shape[0], dtype=float)

low_before = np.zeros(shape=news.shape[0], dtype=float)
low_in_moment = np.zeros(shape=news.shape[0], dtype=float)
low_after = np.zeros(shape=news.shape[0], dtype=float)

high_before = np.zeros(shape=news.shape[0], dtype=float)
high_in_moment = np.zeros(shape=news.shape[0], dtype=float)
high_after = np.zeros(shape=news.shape[0], dtype=float)

for i in range(news.shape[0]):

  for j in range(sber_hour.shape[0]):

    if datetime.datetime(news.date[i].year, news.date[i].month, news.date[i].day, news.date[i].hour) - sber_hour.date[j] == datetime.timedelta(0):

        open_before[i] = sber_hour.open[i-1]
        open_in_moment[i] = sber_hour.open[i]
        open_after[i] = sber_hour.open[i+1]

        close_before[i] = sber_hour.close[i-1]
        close_in_moment[i] = sber_hour.close[i]
        close_after[i] = sber_hour.close[i+1]

        low_before[i] = sber_hour.low[i-1]
        low_in_moment[i] = sber_hour.low[i]
        low_after[i] = sber_hour.low[i+1]

        high_before[i] = sber_hour.high[i-1]
        high_in_moment[i] = sber_hour.high[i]
        high_after[i] = sber_hour.high[i+1]

In [ ]:
news['open_before'] = open_before
news['open_in_moment'] = open_in_moment
news['open_after'] = open_after

news['close_before'] = close_before
news['close_in_moment'] = close_in_moment
news['close_after'] = close_after

news['low_before'] = low_before
news['low_in_moment'] = low_in_moment
news['low_after'] = low_after

news['high_before'] = high_before
news['high_in_moment'] = high_in_moment
news['high_after'] = high_after

In [ ]:
target_open = np.zeros(news.shape[0])

for i in range(news.shape[0]):
  if (news.open_after[i] - news.open_in_moment[i]) > 0:
    target_open[i] = 1
  elif (news.open_after[i] - news.open_in_moment[i]) < 0:
    target_open[i] = -1

news['target_open'] = target_open

# Протестируем небольшой датасет 

In [ ]:
df = news[['text', 'header', 'target_open']]
print('итого из одного источника за год ', df.replace(0, np.nan).dropna().shape[0], 'полезных новостей')
# для обучения и теста это очень мало, но для построения бейзлайна вполне сойдет, сейчас я сделаю модель, которую хоть как-то нужно затестить, а потом соберу нормальные данные

In [ ]:
df.to_csv('sber_196_news_with_target.csv')

In [19]:
import numpy as np

df = pd.read_csv('sber_196_news_with_target.csv').drop('Unnamed: 0', axis=1).replace(0, np.nan).dropna().replace(-1, 0)
df = df.reset_index(drop=True)

In [74]:
df.head()

,text,header,target_open
0,рынок акций рф во вторник вырос вслед за миро...,рынок акций рф во вторник вырос вслед за миром...,0.0
1,рынок акций начал неделю с роста выше 2800п п...,рынок акций начал неделю с роста выше 2800п по...,1.0
2,троценко стал контролирующим акционером geopr...,троценко стал контролирующим акционером geopro...,0.0
3,"""сбермаркет"" начал подключение непродовольств...","""сбермаркет"" начал подключение непродовольстве...",0.0
4,"афк ""система"" вывела из залога в сбербанке 6,...","афк ""система"" вывела из залога в сбербанке 6,8...",0.0


In [21]:
# для начала посмотрю, можно ли решить задачу простыми способами без лишни наворочек
from sklearn.feature_extraction.text import CountVectorizer as CountVectorizer

count_vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=3000)
X = count_vectorizer.fit_transform(df.text)

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split as tt
from xgboost import XGBClassifier
from sklearn.svm import SVC

train_data, test_data, train_target, test_target = tt (X, df.target_open, test_size=0.25)
log_reg = LogisticRegression()
xgb_clf = XGBClassifier()
svc = SVC(kernel='rbf')

xgb_clf.fit(train_data, train_target)
log_reg.fit(train_data, train_target)
svc.fit(train_data, train_target)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [23]:
from sklearn.metrics import accuracy_score, roc_auc_score

1 - accuracy_score(test_target, xgb_clf.predict(test_data))
#roc_auc_score(test_target, xgb_clf.predict(test_data))

0.5510204081632653

In [24]:
1 - accuracy_score(test_target, log_reg.predict(test_data))

0.5714285714285714

In [216]:
1 - accuracy_score(test_target, svc.predict(test_data))

0.6122448979591837

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier as rand_for_clf

vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=4000)
X = vectorizer.fit_transform(df.text)
train_data, test_data, train_target, test_target = tt (X, df.target_open, test_size=0.25)

log_reg.fit(train_data, train_target)
 
accuracy_score(test_target, log_reg.predict(test_data))

0.4897959183673469

# BERT

In [31]:
df.head()

,text,header,target_open
0,рынок акций рф во вторник вырос вслед за миро...,рынок акций рф во вторник вырос вслед за миром...,0.0
1,рынок акций начал неделю с роста выше 2800п п...,рынок акций начал неделю с роста выше 2800п по...,1.0
2,троценко стал контролирующим акционером geopr...,троценко стал контролирующим акционером geopro...,0.0
3,"""сбермаркет"" начал подключение непродовольств...","""сбермаркет"" начал подключение непродовольстве...",0.0
4,"афк ""система"" вывела из залога в сбербанке 6,...","афк ""система"" вывела из залога в сбербанке 6,8...",0.0


In [28]:
!pip install transformers

     |████████████████████████████████| 778kB 3.3MB/s 
     |████████████████████████████████| 3.0MB 15.3MB/s 
     |████████████████████████████████| 890kB 38.4MB/s 
     |████████████████████████████████| 1.1MB 45.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=dd2aae4ac585679ff27631e2b89c12e6167b8182e75a858fae52e5a674ecee95
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [53]:
from sklearn.model_selection import GridSearchCV
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [30]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, 
                            ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [38]:
for i in range(df.shape[0]): # проблема - слишком длинные новости, ее длина в среднем 2000, а для токенайзера максимум 512
# Решение - разбивать на куски длиной не более 512 (конец - конец предложения) и тональность самой новости считать как среднее из тональностей ее составляющих
   df['text'][i] =  df['text'][i][:512]

In [42]:
tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [43]:
tokenized

0      [101, 1195, 29113, 18947, 14150, 23925, 1180, ...
1      [101, 1195, 29113, 18947, 14150, 23925, 1180, ...
2      [101, 1197, 16856, 14150, 29751, 15290, 18947,...
3      [101, 1000, 1196, 29740, 15290, 16856, 29745, ...
4      [101, 1180, 29749, 23925, 1000, 1196, 10325, 2...
                             ...                        
191    [101, 1189, 29748, 29742, 16856, 10325, 18947,...
192    [101, 1194, 14150, 29746, 29113, 22919, 28598,...
193    [101, 4372, 1009, 1182, 29113, 23925, 29748, 2...
194    [101, 1194, 16856, 10260, 25529, 10325, 22919,...
195    [101, 1191, 10325, 18947, 29756, 23925, 14150,...
Name: text, Length: 196, dtype: object

In [75]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(196, 432)

In [48]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [49]:
features = last_hidden_states[0][:,0,:].numpy()

In [67]:
xgb_score = []
log_score = []

N = 100
for i in range(N):
  train_features, test_features, train_labels, test_labels = tt(features,
                                  df['target_open'], random_state=i*5)
  
  log_reg.fit(train_features, train_labels)
  xgb_clf.fit(train_features, train_labels)

  xgb_score.append(1 - accuracy_score(test_labels, xgb_clf.predict(test_features)))
  log_score.append(1 - accuracy_score(test_labels, log_reg.predict(test_features)))

In [68]:
np.mean(xgb_score)

0.5506122448979591

In [69]:
np.mean(log_score)

0.5412244897959184